In [1]:
from src.ingest import *
from src.search import *
import time
import pandas as pd

In [2]:
cols =  ['database', 'chunk_size', 'overlap', 'clean', 'embedding', 'chunks_processed', 'time_to_process']
results = pd.DataFrame(columns = cols)

In [3]:

embedding_model = 'nomic-embed-text'
db = 'redis'

for chunk in [100,300,500,1000]:
    for overlap in [0, 50, 100]:
        for clean in [True, False]:
            clear_redis_store()

            start = time.time()
            create_hnsw_index()
            chunk_count = process_pdfs("data/", chunk_size=chunk, overlap=overlap, clean = clean)

            to_complete = time.time() - start
            print(f'Index with chunk size {chunk} and overlap {overlap} created in {round(to_complete, 2)} seconds')

            new_row = [db, chunk, overlap, clean, embedding_model, chunk_count, to_complete]

            results = pd.concat([results, pd.DataFrame([new_row], columns=cols)], axis = 0)





Index with chunk size 100 and overlap 0 created in 8.54 seconds


/var/folders/d4/c154mt9s11x5q965j91l06qw0000gn/T/ipykernel_29012/2260902380.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame([new_row], columns=cols)], axis = 0)


Index with chunk size 100 and overlap 0 created in 8.67 seconds


KeyboardInterrupt: 